In [1]:
import pandas as pd
import numpy as np

In [7]:
movies = pd.read_csv('./movielens/movies.csv')
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null int64
title      27278 non-null object
genres     27278 non-null object
dtypes: int64(1), object(2)
memory usage: 639.4+ KB


In [6]:
ratings = pd.read_csv('./movielens/ratings.csv')
print(ratings.shape)

(20000263, 4)


In [8]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


# Step 1: Calculate average rating and number of rated users for each movie

In [10]:
ratings['rating'].isnull().any()

False

In [11]:
grouped = ratings.groupby('movieId')
movies_rating = grouped['rating'].agg([np.size, np.mean, np.std])
print(movies_rating.shape)
print(movies_rating.dtypes)

(26744, 3)
size    float64
mean    float64
std     float64
dtype: object


In [16]:
movies_rating.tail(10)

,index,movieId,n_users,rating_mean,rating_std
26734,26734,131241,1.0,4.0,NaN
26735,26735,131243,1.0,4.0,NaN
26736,26736,131248,1.0,4.0,NaN
26737,26737,131250,1.0,4.0,NaN
26738,26738,131252,1.0,4.0,NaN
26739,26739,131254,1.0,4.0,NaN
26740,26740,131256,1.0,4.0,NaN
26741,26741,131258,1.0,2.5,NaN
26742,26742,131260,1.0,3.0,NaN
26743,26743,131262,1.0,4.0,NaN


In [14]:
movies_rating = movies_rating.reset_index()
movies_rating.head(10)

,index,movieId,size,mean,std
0,0,1,49695.0,3.921240,0.889012
1,1,2,22243.0,3.211977,0.951150
2,2,3,12735.0,3.151040,1.006642
3,3,4,2756.0,2.861393,1.095702
4,4,5,12161.0,3.064592,0.982140
5,5,6,23899.0,3.834930,0.880468
6,6,7,12961.0,3.366484,0.955883
7,7,8,1415.0,3.142049,0.966353
8,8,9,3960.0,3.004924,0.960203
9,9,10,29005.0,3.430029,0.859035


In [15]:
movies_rating = movies_rating.rename(columns={'size':'n_users', 'mean':'rating_mean', 'std':'rating_std'})
movies_rating.head(10)

,index,movieId,n_users,rating_mean,rating_std
0,0,1,49695.0,3.921240,0.889012
1,1,2,22243.0,3.211977,0.951150
2,2,3,12735.0,3.151040,1.006642
3,3,4,2756.0,2.861393,1.095702
4,4,5,12161.0,3.064592,0.982140
5,5,6,23899.0,3.834930,0.880468
6,6,7,12961.0,3.366484,0.955883
7,7,8,1415.0,3.142049,0.966353
8,8,9,3960.0,3.004924,0.960203
9,9,10,29005.0,3.430029,0.859035


In [17]:
movies_rating['n_users'] = movies_rating['n_users'].astype(int)

In [18]:
movies_rating.head(10)

,index,movieId,n_users,rating_mean,rating_std
0,0,1,49695,3.921240,0.889012
1,1,2,22243,3.211977,0.951150
2,2,3,12735,3.151040,1.006642
3,3,4,2756,2.861393,1.095702
4,4,5,12161,3.064592,0.982140
5,5,6,23899,3.834930,0.880468
6,6,7,12961,3.366484,0.955883
7,7,8,1415,3.142049,0.966353
8,8,9,3960,3.004924,0.960203
9,9,10,29005,3.430029,0.859035


In [19]:
movies_rating['rating_std'] = movies_rating['rating_std'].fillna(0)
movies_rating.tail()

,index,movieId,n_users,rating_mean,rating_std
26739,26739,131254,1,4.0,0.0
26740,26740,131256,1,4.0,0.0
26741,26741,131258,1,2.5,0.0
26742,26742,131260,1,3.0,0.0
26743,26743,131262,1,4.0,0.0


# Step 2: Merging

In [20]:
movies_all = movies.merge(movies_rating, on='movieId', how='left')

In [21]:
movies_all.head(15)

,movieId,title,genres,index,n_users,rating_mean,rating_std
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.0,49695.0,3.921240,0.889012
1,2,Jumanji (1995),Adventure|Children|Fantasy,1.0,22243.0,3.211977,0.951150
2,3,Grumpier Old Men (1995),Comedy|Romance,2.0,12735.0,3.151040,1.006642
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0,2756.0,2.861393,1.095702
4,5,Father of the Bride Part II (1995),Comedy,4.0,12161.0,3.064592,0.982140
5,6,Heat (1995),Action|Crime|Thriller,5.0,23899.0,3.834930,0.880468
6,7,Sabrina (1995),Comedy|Romance,6.0,12961.0,3.366484,0.955883
7,8,Tom and Huck (1995),Adventure|Children,7.0,1415.0,3.142049,0.966353
8,9,Sudden Death (1995),Action,8.0,3960.0,3.004924,0.960203
9,10,GoldenEye (1995),Action|Adventure|Thriller,9.0,29005.0,3.430029,0.859035


In [22]:
movies_all['n_users'] = movies_all['n_users'].fillna(0).astype(int)
movies_all['rating_mean'] = movies_all['rating_mean'].fillna(0)
movies_all['rating_std'] = movies_all['rating_std'].fillna(0)

# Output: Dataframe with added columns

In [23]:
movies_all.head(15)

,movieId,title,genres,index,n_users,rating_mean,rating_std
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.0,49695,3.921240,0.889012
1,2,Jumanji (1995),Adventure|Children|Fantasy,1.0,22243,3.211977,0.951150
2,3,Grumpier Old Men (1995),Comedy|Romance,2.0,12735,3.151040,1.006642
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0,2756,2.861393,1.095702
4,5,Father of the Bride Part II (1995),Comedy,4.0,12161,3.064592,0.982140
5,6,Heat (1995),Action|Crime|Thriller,5.0,23899,3.834930,0.880468
6,7,Sabrina (1995),Comedy|Romance,6.0,12961,3.366484,0.955883
7,8,Tom and Huck (1995),Adventure|Children,7.0,1415,3.142049,0.966353
8,9,Sudden Death (1995),Action,8.0,3960,3.004924,0.960203
9,10,GoldenEye (1995),Action|Adventure|Thriller,9.0,29005,3.430029,0.859035


In [128]:
not_rated = movies_all[movies_all['n_users']==0]
not_rated

,movieId,title,genres,n_users,rating_mean,rating_std
8555,26018,Chase a Crooked Shadow (1958),Crime|Film-Noir|Mystery|Thriller,0,0.0,0.0
8933,26580,"Park Is Mine, The (1986)",Action|Drama|Thriller,0,0.0,0.0
9249,27249,"Trumpet of the Swan, The (2001)",Animation|Drama|Musical,0,0.0,0.0
9315,27396,"Gentleman's Game, A (2002)",Drama,0,0.0,0.0
9770,31797,White Banners (1938),Drama,0,0.0,0.0
9975,32773,Parenti serpenti (1992),Comedy,0,0.0,0.0
10027,33019,Barefoot (Barfuss) (2005),Comedy|Drama|Romance,0,0.0,0.0
10070,33229,"Angry Silence, The (1960)",Drama,0,0.0,0.0
10128,33573,Wu Tang Master (Tian shi zhuang xie) (1983),Action,0,0.0,0.0
11078,45994,National Lampoon's Cattle Call (Cattle Call) (...,Comedy,0,0.0,0.0


# Step 3: Export to csv files

In [25]:
movies_all.to_csv('./movielens/movies_all.csv')